# DataFrameIt - Exemplo 08: Suporte a Múltiplos Tipos de Dados

Este notebook demonstra como usar o DataFrameIt com diferentes tipos de dados além de DataFrames: listas, dicionários e Series.

**Conceitos demonstrados:**
- Processamento de listas de textos
- Processamento de dicionários {chave: texto}
- Processamento de pandas.Series
- Retorno de resultados como DataFrame

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bdcdo/dataframeit/blob/main/example/07_multiple_data_types.ipynb)

## 1. Instalação

In [ ]:
!pip install -q dataframeit[google]

## 2. Configuração da API Key

In [ ]:
import os

try:
    from google.colab import userdata
    os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
    print("API Key carregada dos Secrets do Colab")
except:
    pass

# os.environ['GOOGLE_API_KEY'] = 'sua-chave-aqui'

if 'GOOGLE_API_KEY' not in os.environ:
    print("AVISO: Configure sua GOOGLE_API_KEY antes de continuar")
else:
    print("API Key configurada com sucesso!")

## 3. Imports

In [ ]:
from pydantic import BaseModel, Field
from typing import Literal
import pandas as pd
from dataframeit import dataframeit

## 4. Definir Modelo Pydantic

In [ ]:
class SentimentAnalysis(BaseModel):
    """Estrutura para análise de sentimento."""

    sentimento: Literal['positivo', 'negativo', 'neutro'] = Field(
        ...,
        description="Classificação do sentimento"
    )

    resumo: str = Field(
        ...,
        description="Resumo em uma frase"
    )

TEMPLATE = """
Analise o sentimento do texto abaixo:

{texto}

Classifique como positivo, negativo ou neutro e forneça um breve resumo.
"""

## 5. Processando uma Lista de Textos

Você pode passar uma lista de strings diretamente - não precisa especificar `text_column`!

In [ ]:
textos_lista = [
    "Adorei o produto! Chegou rápido e funciona perfeitamente.",
    "Péssimo atendimento, nunca mais compro aqui.",
    "O produto é ok, nada de especial.",
]

print("Entrada (list):")
for i, texto in enumerate(textos_lista):
    print(f"  [{i}] {texto[:50]}...")

In [ ]:
# Processa a lista diretamente - retorna DataFrame
resultado_lista = dataframeit(
    textos_lista,        # Lista de textos (não precisa de text_column!)
    SentimentAnalysis,
    TEMPLATE
)

print("\nSaída (DataFrame):")
print(f"Tipo de saída: {type(resultado_lista).__name__}")
resultado_lista

## 6. Processando um Dicionário

Ao passar um dicionário, as chaves se tornam o índice do DataFrame resultante.

In [ ]:
textos_dict = {
    'review_001': "Produto excelente! Superou minhas expectativas.",
    'review_002': "Muito ruim, veio quebrado e não consegui trocar.",
    'review_003': "Atende ao esperado, bom custo-benefício.",
}

print("Entrada (dict):")
for key, value in textos_dict.items():
    print(f"  [{key}] {value[:40]}...")

In [ ]:
# Processa o dicionário - retorna DataFrame com chaves como índice
resultado_dict = dataframeit(
    textos_dict,         # Dicionário (não precisa de text_column!)
    SentimentAnalysis,
    TEMPLATE
)

print("\nSaída (DataFrame com chaves como índice):")
print(f"Tipo de saída: {type(resultado_dict).__name__}")
print(f"Índices: {list(resultado_dict.index)}")
resultado_dict

## 7. Processando uma Pandas Series

O índice da Series é preservado no DataFrame resultante.

In [ ]:
textos_series = pd.Series(
    [
        "Fantástico! A melhor compra que já fiz.",
        "Decepcionado com a qualidade.",
        "Normal, como qualquer outro produto similar.",
    ],
    index=['cliente_A', 'cliente_B', 'cliente_C'],
    name='avaliacoes'
)

print("Entrada (pd.Series):")
print(textos_series)

In [ ]:
# Processa a Series - retorna DataFrame preservando o índice
resultado_series = dataframeit(
    textos_series,       # Series (não precisa de text_column!)
    SentimentAnalysis,
    TEMPLATE
)

print("\nSaída (DataFrame preservando índice):")
print(f"Tipo de saída: {type(resultado_series).__name__}")
print(f"Índices preservados: {list(resultado_series.index)}")
resultado_series

## 8. Comparação com DataFrame Tradicional

In [ ]:
# Forma tradicional com DataFrame
df_tradicional = pd.DataFrame({
    'id': [1, 2, 3],
    'texto': textos_lista
})

resultado_df = dataframeit(
    df_tradicional,
    SentimentAnalysis,
    TEMPLATE,
    text_column='texto'  # Necessário para DataFrame
)

print("DataFrame tradicional:")
resultado_df

## 9. Resumo dos Tipos de Dados

| Tipo de Entrada | `text_column` | Tipo de Saída |
|-----------------|---------------|-------------------|
| `pd.DataFrame` | Obrigatório (padrão: `'texto'`) | `pd.DataFrame` com colunas originais + extraídas |
| `pl.DataFrame` | Obrigatório (padrão: `'texto'`) | `pl.DataFrame` com colunas originais + extraídas |
| `list` | Automático | `pd.DataFrame` (índice numérico) |
| `dict` | Automático | `pd.DataFrame` (chaves como índice) |
| `pd.Series` | Automático | `pd.DataFrame` (índice preservado) |
| `pl.Series` | Automático | `pl.DataFrame` |

---

## Próximos Passos

- [01_basic.ipynb](01_basic.ipynb) - Voltar ao básico
- [rate_limiting.ipynb](rate_limiting.ipynb) - Configurar rate limiting